In [ ]:
import pandas as pd
hist = pd.read_csv("/Users/jaeho/Work/yeoboya/recommender/data/tabula/m_yeoboya_ai_dv_mem_item_use_hist.csv")
len(hist)

In [ ]:
valid_code = {
    "mem_concn",
    "msg_10day",
    "msg_15day",
    "msg_1day",
    "msg_30day",
    "msg_30day_2",
    "msg_5day",
    "msg_o1hr",
    "pf_view",
    "ps_view_10",
    "ps_view_15",
    "ps_view_30",
    "ps_view_5",
    "vo_mt"
}

In [ ]:
from collections import defaultdict

user_rating = defaultdict(dict)

for _, row in hist.iterrows():
    if row['item_code'] in valid_code:
        if row['ptr_mem_no'] in user_rating[row['mem_no']]:
            if user_rating[row['mem_no']][row['ptr_mem_no']] < 10:
                user_rating[row['mem_no']][row['ptr_mem_no']] += 2
        else:
            user_rating[row['mem_no']][row['ptr_mem_no']] = 2
        

In [ ]:
######################### WRITING DATA #########################
import pickle
with open('./user_rating_test_data.pkl', 'wb') as f:
    pickle.dump(user_rating, f)

In [ ]:
import pandas as pd
data = pd.read_csv('/Users/jaeho/Work/yeoboya/recommender/data/tabula/member_mate.csv')

In [ ]:
data.shape

In [ ]:
import pickle
with open('/Users/jaeho/Work/yeoboya/recommender/data/pkl_files/user_rating_test_data.pkl', 'rb') as f:
    user_rating = pickle.load(f)
rated_user = set()
rated_user2 = set()
rated_user3 = set()


for k, v in user_rating.items():
    rated_user.add(k)
    rated_user3.add(k)
    for u,r in v.items():
        rated_user2.add(u)
        rated_user3.add(u)

print(len(rated_user))
print(len(rated_user2))
print(len(rated_user3))

In [ ]:
new_user_df = pd.DataFrame()

for idx, row in data.iterrows():
    if row['mem_no'] in rated_user3:
        new_user_df = pd.concat([new_user_df, pd.DataFrame(row).T])

In [ ]:
new_user_df.shape

In [ ]:
mem_id = set(new_user_df['mem_no'].values)
len(mem_id)

In [ ]:
import pandas as pd
data = pd.read_csv('/Users/jaeho/Work/yeoboya/recommender/data/tabula/m_yeoboya_ai_dv_member_mate_exit.csv')

In [ ]:
exit_user_df = pd.DataFrame()

for idx, row in data.iterrows():
    if row['mem_no'] in rated_user3 and row['mem_no'] not in mem_id:
        exit_user_df = pd.concat([exit_user_df, pd.DataFrame(row).T])

In [ ]:
exit_user_df.shape

In [ ]:
new_data_frame = pd.concat([new_user_df, exit_user_df])
print(new_data_frame.shape)
new_data_frame.head()

In [ ]:
new_data_frame = new_data_frame.drop(columns=["chrgr_name", "exit_date"])

In [ ]:
train_mem_id = set(list(map(int, new_data_frame['mem_no'].tolist())))
print(len(train_mem_id))

In [ ]:
import pickle
with open('/Users/jaeho/Work/yeoboya/recommender/data/pkl_files/user_rating_test_data.pkl', 'rb') as f:
    user_rating_data = pickle.load(f)

In [ ]:
print(len(user_rating_data))

In [ ]:
from collections import defaultdict
filtered_user_rating = defaultdict(dict)

for k, v in user_rating_data.items():
    if k in train_mem_id:
        for u, r in v.items():
            if u in train_mem_id:
                filtered_user_rating[k][u] = r

print(len(filtered_user_rating))

In [ ]:
######################### WRITING DATA #########################
import pickle
with open("label_user_rating.pkl", "wb") as f:
    pickle.dump(filtered_user_rating, f)

In [ ]:
import pickle
import pandas as pd

with open("/Users/jaeho/Work/yeoboya/recommender/data/nn_data/label/label_user_rating.pkl", "rb") as f:
    user_rating = pickle.load(f)
user_info = pd.read_csv("/Users/jaeho/Work/yeoboya/recommender/data/nn_data/train/train_user_data.csv")

In [ ]:
print(len(user_rating))
print(user_info.shape)
user_info.head()

In [ ]:
print(len(user_rating))

In [ ]:
user_id_dict = {}
used_id_set = set()

id = 0
for user_id, target in user_rating.items():
    if user_id not in user_id_dict:
        user_id_dict[user_id] = id
        used_id_set.add(id)
        id += 1
        
    for ptr_id, rating in target.items():
        if ptr_id not in user_id_dict:
            user_id_dict[ptr_id] = id
            used_id_set.add(id)
            id += 1

print(len(used_id_set))
print(len(user_id_dict))

In [ ]:
from collections import defaultdict

new_rating = defaultdict(dict)

for user_id, target in user_rating.items():
    for ptr_id, rating in target.items():
        new_rating[user_id_dict[user_id]][user_id_dict[ptr_id]] = rating
print(len(new_rating))

In [ ]:
######################### WRITING DATA #########################
import pickle

with open("new_user_rating.pkl", "wb") as f:
    pickle.dump(new_rating, f)

In [ ]:
import pandas as pd
user_info = pd.read_csv("/Users/jaeho/Work/yeoboya/recommender/data/nn_data/train/train_user_data.csv")
user_info.head()

In [ ]:
for i, row in user_info.iterrows():
    user_info.loc[i, "mem_no"] = user_id_dict[row["mem_no"]]
user_info.head()

In [ ]:
######################### WRITING DATA #########################
user_info.to_csv("new_user_data.csv", index=False)

In [ ]:
user_info.shape